In [45]:
import os
import time
from glob import glob
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import cv2
import shutil
import subprocess


# from mpire import WorkerPool
import tensorflow as tf

In [46]:
similarity_model = tf.keras.models.load_model("model")
similarity_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 lambda (Lambda)             (None, 1024)              0         
                                                                 
Total params: 3228864 (12.32 MB)
Trainable params: 3206976 (12.23 MB)
Non-trainable params: 21888 (85.50 KB)
_________________________________________________________________


In [47]:
def watermark_check(sample, base, filename):
    image_diff = np.abs(sample[...,0] - base[...,0]) / 255
    
    X = np.abs(image_diff).mean(axis=0)
    Y = np.abs(image_diff).mean(axis=1)
    
    # plt.figure()
    # plt.imshow(image_diff/ 255.0)
    # plt.show()
    #plt.figure()
    #plt.plot(X)
    #plt.plot(Y)
    
    x1, x2 = np.where(X > 0.025)[0][[0,-1]]
    y1, y2 = np.where(Y > 0.025)[0][[0,-1]]
    watermark = sample[y1:y2, x1:x2]

    
    shutil.copy(filename, f"image_match_ALL/{os.path.basename(filename)}")
    if abs((watermark.shape[1] / watermark.shape[0]) - 2) > 0.05:
        #target_image = tf.keras.utils.load_img(filename, target_size=(800,800))
        #target_image = tf.keras.utils.img_to_array(target_image)
        #plt.figure()
        #plt.imshow(target_image / 255.0)
        #plt.show()
        shutil.copy(filename, f"image_match/{os.path.basename(filename)}")
        #cv2.imwrite(f"image_match/{os.path.basename(filename)}", watermark.astype(np.uint8))

In [48]:
@tf.function
def inference(x):
    return similarity_model(x)

In [49]:
base = tf.keras.utils.load_img("base.webp", target_size=(400,400))
base = tf.keras.utils.img_to_array(base)

base_ = cv2.resize(base.copy(), (224,224))
feature_base = inference(base_[None] / 255)[0]


def detect_ganj(filename):
    try:
        sample = tf.keras.utils.load_img(filename, target_size=(400,400))
        sample = tf.keras.utils.img_to_array(sample)
    
        sample_ = cv2.resize(sample.copy(), (224,224))
    
        feature_sample = inference(sample_[None] / 255)[0]
        similarity = np.sum(feature_sample * feature_base)
        if similarity > 0.6:
            #shutil.copy(filename, f"image_match/{os.path.basename(filename)}")
            watermark_check(sample, base, filename)
    except Exception as e:
        # print("Error in detect_ganj:",e)
        # subprocess.Popen(f'explorer /select,"{filename}"')
        print("ProductID=", os.path.basename(filename).split("_")[0])
        pass

In [50]:
base_filenames = []

In [51]:
start = time.time()
time.sleep(2)
while True:
    end = time.time()
    time_diff = end - start
    
    if time_diff > 1:
        start = time.time()
        filenames = glob("images/*.jpg")
        diff_filenames = list(set(filenames) - set(base_filenames))
        if len(diff_filenames):
            #with WorkerPool(n_jobs=4) as pool:
            #    results = pool.map(detect_ganj, diff_filenames, progress_bar=True)

            
            for filename in diff_filenames:
                detect_ganj(filename)
        
        base_filenames = filenames

ProductID= 1577639
ProductID= 1577639
ProductID= 1675102
ProductID= 1435580
ProductID= 1675102
ProductID= 1675102
ProductID= 1435580
ProductID= 1675102
ProductID= 1693858
ProductID= 9125912
ProductID= 9256850
ProductID= 6939999
ProductID= 7423705
ProductID= 5519653


KeyboardInterrupt: 